In [20]:
# Libraries 
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
# import sys
# import os
# sys.path.append(os.path.abspath(os.path.join('..', 'src'))) 


# Configuration
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)


### Phase 1: Exploration and Cleanup

In [21]:
# Exploratory Data Analysis
    # create dataframes DONE
    # at dataframe level 
        # print df name DONE
        # visual exploration with .head(), .tail() and sample() DONE
        # dataframe overview with .info() DONE
        # check duplicates with .duplicated().sum DONE
            # create separate dataframe with duplicates if needed DONE
        # check null values .isnull().sum() DONE
        # explore main statistical metrics for numerical and non numerical columns with .describe() and .describe(include = "object")
    # at column level
        # review columns names with .columns DONE
        # review unique values with .unique() DONE
        # review value count with .value_counts() DONE
        # check null values with .isnull().sum() DONE
        # atypical values
    # create function/s for the EDA
# Transformation
# Dataframe union


In [22]:
# 📂 Open data 

df_customer_flights = pd.read_csv("data/customer_flight_activity.csv")
df_customer_loyalty = pd.read_csv("data/customer_loyalty_history.csv")
dataframes_dict = {"df_customer_flight✈️" : df_customer_flights, "df_customer_loyalty💞" : df_customer_loyalty}

In [64]:
# ✍️ Exploratory Data Analysis functions definition

def explore_dataframes (df_dict):
    """
    Provides relevant information for a Exploratory Data Analysis.
    
    This function receives a dict with dataframes and executes a series of 
    functions aiming to provide relevant information to analize the data.

    Args:

    Returns:
    
    """
    duplicate_dataframes_dict = {}
    for name, df in df_dict.items():
        df_name = f"'{name.upper().replace('_',' ')}'"
        print(f" \n\n----------- DATAFRAME NAME: {df_name} -----------")
        print(f"\n{df_name} ---> Dataframe INFO:\n")
        display(df.info())
        print(f"\n{df_name} ---> FIRST FIVE (5) ROWS:")
        display(df.head())
        print(f"\n{df_name} ---> LAST FIVE (5) ROWS:")
        display(df.tail())
        print(f"\n{df_name} ---> SAMPLE (5) ROWS:")
        display(df.sample(5))
        print(f" \n{df_name} ---> DUPLICATES COUNT IS: {df.duplicated().sum()}, {round((df.duplicated().sum()/df.shape[0]*100),2)}% OVER TOTAL ROWS\n")
        if df.duplicated().sum() > 0:
            duplicates_df_name = name + "_duplicates"
            duplicates_df = df[df.duplicated(keep=False)]
            print(f"{df_name} ---> DATAFRAME WITH DUPLICATED ROWS (INCLUDING ALL APPEARANCES):\n")
            display(duplicates_df.head(10))
            duplicate_dataframes_dict[duplicates_df_name] = duplicates_df
        print(f"\n{df_name} --> COUNT OF ROWS WITH ALL NULL VALUES IS: {df.isnull().all(axis=1).sum()}\n")
        print(f"\n{df_name} --> COUNT OF COLUMNS WITH ALL NULL VALUES IS: {df.isnull().all().sum()}\n")
        print(F"\n{df_name} --> STATISTICAL METRICS FOR NUMERICAL COLUMNS:")
        display(df.describe().T)
        try:
            print(F"\n{df_name} --> STATISTICAL METRICS FOR CATEGORICAL COLUMNS:")
            display(df.describe(include="object").T)
        except:
            print("\nUPS... IT SEEMS LIKE THERE ARE NO COLUMNS WITH CATEGORICATL DATA")
    return duplicate_dataframes_dict

def explore_columns (df_dict):
    for name, df in df_dict.items():
        df_name = f"'{name.upper().replace('_',' ')}'"
        print(f" \n\n----------- DATAFRAME NAME: {df_name} -----------")
        for index, column in enumerate(df.columns):
            print (f"\n{index}) Column {column.upper()} (from {(df_name)} dataframe):")
            print (f"\n>>> UNIQUE VALUES:")
            print (df[column].unique())
            print (f"\n>>> VALUES COUNT:")
            print (df[column].value_counts())
            print (f"\n>>> COUNT OF DUPLICATES IN THE COLUMN:")
            print (df.duplicated(subset=[column]).sum())
            print (f"\n>>> COUNT OF NULL VALUES IN THE COLUMN:")
            print (df[column].isnull().sum())
            if df[column].dtype in ['int64', 'float64']:
                print("\nSTATISTICAL DESCRIPTION (NUMERIC):")
                display(df[column].describe())
            else:
                print("\nSTATISTICAL DESCRIPTION (CATEGORICAL):")
                display(df[column].describe(include='object'))
            print ("--------")

In [25]:
# ▶️ Exploratory Data Analysis code execution

explore_dataframes(dataframes_dict)

explore_columns(dataframes_dict)



'DF CUSTOMER FLIGHT✈️' ---> Dataframe COLUMNS:

Index(['loyalty_number', 'year', 'month', 'flights_booked',
       'flights_with_companions', 'total_flights', 'distance',
       'points_accumulated', 'points_redeemed', 'dollar_cost_points_redeemed'],
      dtype='object')

'DF CUSTOMER LOYALTY💞' ---> Dataframe COLUMNS:

Index(['loyalty_number', 'country', 'province', 'city', 'postal_code',
       'gender', 'education', 'salary', 'marital_status', 'loyalty_card',
       'clv', 'enrollment_type', 'enrollment_year', 'enrollment_month',
       'cancellation_year', 'cancellation_month'],
      dtype='object')


In [76]:
# ✍️ Data transformation (cleaning and union) functions definition

def columns_to_snake_case (df_dict):
    for name, df in df_dict.items():
        df_name = f"'{name.upper().replace('_',' ')}'"
        df.columns = df.columns.str.lower().str.replace(' ', '_')
        print(f"\n{df_name} ---> Dataframe COLUMNS:\n")
        print(df.columns)

def impute_nulls_as_special_category(df, column_list, category_name):
    # Iterate through the list of columns to replace nulls with "Unknown"
    for column in column_list:
        if column in df.columns:
            # Replace nulls with the value "Unknown" for each column in the list
            df[column] = df[column].fillna(category_name)
            print (f"\nNull values imputed ✅ in column {column}.")
            print (f"UNIQUE VALUES:")
            print (df[column].unique())
        else:
            print(f"❌ The column '{column}' does not exist in the DataFrame.")

def transform_negative_values(df,column_list):
    """Transform negative values into its absolute value"""
    for column in column_list:
        if column in df.columns:
            df[column] = df[column].abs()
            print (f"\nNegative values transformed ✅ in column {column}.")
            print (f"UNIQUE VALUES:")
            print (df[column].unique())
        else:
            print(f"❌ The column '{column}' does not exist in the DataFrame.")

def impute_nulls_as_median(df, column_list):
    # Iterate through the list of columns to replace nulls with median
    for column in column_list:
        if column in df.columns:
            median= df[column].median()
            # Replace nulls with the median for each column in the list
            df[column] = df[column].fillna(median)
            print (f"\nNull values imputed ✅ in column {column}.")
            print (f"UNIQUE VALUES:")
            print (df[column].unique())
        else:
            print(f"❌ The column '{column}' does not exist in the DataFrame.")

In [78]:
# ▶️ Data Transformation code execution

# Rename columns to snake case format
columns_to_snake_case(dataframes_dict)

# In df customer_flights, delete records that are duplicated in all columns keeping only the first appearance
df_customer_flights.drop_duplicates(inplace=True)

print(f"DUPLICATES COUNT IS: {df_customer_flights.duplicated().sum()}")

# In columns "cancellation_year" and "cancelation_month" replace NaN with "Not Cancelled"

columns = ["cancellation_year","cancellation_month"]
category_name = "Not Cancelled"

impute_nulls_as_special_category(df_customer_loyalty,columns,category_name)

columns = ["salary"]
transform_negative_values(df_customer_loyalty,columns)

impute_nulls_as_median(df_customer_loyalty, columns)

impute_nulls_as_median(df_customer_loyalty, columns)



'DF CUSTOMER FLIGHT✈️' ---> Dataframe COLUMNS:

Index(['loyalty_number', 'year', 'month', 'flights_booked',
       'flights_with_companions', 'total_flights', 'distance',
       'points_accumulated', 'points_redeemed', 'dollar_cost_points_redeemed'],
      dtype='object')

'DF CUSTOMER LOYALTY💞' ---> Dataframe COLUMNS:

Index(['loyalty_number', 'country', 'province', 'city', 'postal_code',
       'gender', 'education', 'salary', 'marital_status', 'loyalty_card',
       'clv', 'enrollment_type', 'enrollment_year', 'enrollment_month',
       'cancellation_year', 'cancellation_month'],
      dtype='object')
DUPLICATES COUNT IS: 0

Null values imputed ✅ in column cancellation_year.
UNIQUE VALUES:
['Not Cancelled' 2018.0 2015.0 2017.0 2014.0 2016.0 2013.0]

Null values imputed ✅ in column cancellation_month.
UNIQUE VALUES:
['Not Cancelled' 1.0 12.0 4.0 2.0 7.0 11.0 5.0 6.0 10.0 8.0 9.0 3.0]

Negative values transformed ✅ in column salary.
UNIQUE VALUES:
[ 83236.  73455. 103495. ...  76178.

# Loyalty number column analysis

In [48]:
def loyalty_number_analysis (df):
    
    # check the number of duplicates in loyalty_number column
    duplicates = df_customer_flights.duplicated(subset=['loyalty_number']).sum()
    print(f"There are {duplicates} duplicates in the column 'loyalty_number', which represent {round((duplicates/df_customer_flights.shape[0]*100),2)}% over the total")
    
    df_loyalty_number_duplicates = df_customer_flights[df_customer_flights.duplicated(subset=['loyalty_number'])]
    print (f"\n>>>DATAFRAME WITH loyalty_number:")
    display(df_loyalty_number_duplicates)
    # the information in the dataframe is organized in a way that there is one row per each year and each month within a year. 
    # Since there is info from 2 years it is expected to have 24 rows per loyalty_number (or less). So, I will only focus the analysis in those that appear more than 24 times
    # Calculate the times of appearance for each loyalty_number
    loyalty_number_value_counts = df_customer_flights['loyalty_number'].value_counts()
    print (f"\n>>> VALUES COUNT:")
    print(loyalty_number_value_counts)
    # filter the Series to keep only the values that appear more than 24 times
    values_to_keep = loyalty_number_value_counts[loyalty_number_value_counts > 24].index
    print (f"\n>>> VALUES WITH MORE THAN 24 ROWS:")
    print(values_to_keep)

There are 388887 duplicates in the column 'loyalty_number', which represent 95.87% over the total


In [55]:
filtered_df = df_customer_flights[df_customer_flights['loyalty_number'].isin(values_to_keep)].sort_values(by=['loyalty_number','year', 'month','total_flights'])
filtered_df.head(20)

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed
41,101902,2017,1,0,0,0,0,0.0,0,0
42,101902,2017,1,0,0,0,0,0.0,0,0
16942,101902,2017,2,0,0,0,0,0.0,0,0
16943,101902,2017,2,0,0,0,0,0.0,0,0
33843,101902,2017,3,0,0,0,0,0.0,0,0
33844,101902,2017,3,0,0,0,0,0.0,0,0
50744,101902,2017,4,4,0,4,1460,146.0,0,0
185796,101902,2017,4,4,4,8,2384,238.0,488,40
67646,101902,2017,5,7,0,7,3318,331.0,0,0
67645,101902,2017,5,9,3,12,2748,274.0,0,0


In [62]:
df_customer_loyalty[df_customer_loyalty['loyalty_number'].isin(values_to_keep)]

,loyalty_number,country,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
5,193662,Canada,Yukon,Whitehorse,Y2K 6R0,Male,Bachelor,51124.0,Married,Star,3844.57,Standard,2012,5,NaN,NaN
123,746226,Canada,British Columbia,Whistler,V6T 1Y8,Female,Bachelor,63501.0,Married,Star,4089.04,Standard,2018,1,NaN,NaN
141,279419,Canada,British Columbia,West Vancouver,V6V 8Z3,Female,College,NaN,Single,Star,4117.37,Standard,2013,7,NaN,NaN
161,354438,Canada,Quebec,Montreal,H2T 2J6,Male,College,NaN,Married,Star,4167.09,Standard,2018,9,NaN,NaN
204,719633,Canada,British Columbia,Victoria,V10 6T5,Male,Bachelor,52109.0,Divorced,Star,4250.78,Standard,2016,12,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14233,411030,Canada,Ontario,Toronto,P1J 8T7,Female,College,NaN,Single,Star,6503.14,Standard,2017,4,NaN,NaN
14263,732304,Canada,Quebec,Quebec City,G1B 3L5,Male,Bachelor,82727.0,Married,Nova,5715.79,2018 Promotion,2018,4,NaN,NaN
14579,243741,Canada,British Columbia,Vancouver,V6E 3D9,Female,Bachelor,93595.0,Married,Star,7425.85,Standard,2013,4,NaN,NaN
14937,897772,Canada,Quebec,Montreal,H2Y 4R4,Male,High School or Below,44490.0,Divorced,Star,8123.96,Standard,2012,9,NaN,NaN
